In [13]:
import numpy as np

def partial_corr(X, Y):
    n = X.shape[0]
    cX = X - np.sum(X, axis=0, keepdims=True) / n
    cY = Y - np.sum(Y, axis=0, keepdims=True) / n
    cov_XY = 1. / (n - 1) * np.dot(cX.T, cY)
    var_X = 1. / (n - 1) * np.sum(cX**2, axis=0)
    var_Y = 1. / (n - 1) * np.sum(cY**2, axis=0)
    return cov_XY / np.sqrt(var_X[:, None] * var_Y[None, :])

def mean_center_scale(x):
    x = x - x.mean(0)
    return x / x.std(0)

class SCCA:
    """Implementation of sparce canonical correlation
    Reference: Parkhomenko et al.: Sparse Canonical Correlation Analysis
    This is a work in progress, not expected to work yet.
    """
    def __init__(self, X, Y, c1=0.01, c2=0.01, lu=0.0001, lv=0.0002):
        self.X = mean_center_scale(X)
        self.Y = mean_center_scale(Y)
        self.p = np.shape(X)[1]
        self.q = np.shape(Y)[1]
        self.c1 = c1
        self.c2 = c2
        self.lu = lu
        self.lv = lv

    def _K(self):
        xx = partial_corr(self.X, self.X)
        yy = partial_corr(self.Y, self.Y)
        xy = partial_corr(self.X, self.Y)
        #xxi = np.linalg.inv(np.diagxx)
        #yyi = np.linalg.inv(yy)
        xxi = np.linalg.inv(np.diag(np.diag(xx)))
        yyi = np.linalg.inv(np.diag(np.diag(yy)))
        return np.dot(xxi, xy).dot(yyi)

    def _norm(self, w):
        lw=np.sqrt(np.dot(w, w))
        return w/lw

    def _soft_thresh(self, w, t):
        if t == 'u':
            w = (np.abs(w) - .5*self.lu) + np.sign(w)
        elif t == 'v':
            w = (np.abs(w) - .5*self.lv) + np.sign(w)
        return w

    def fit(self):
        u = np.random.sample(self.p)
        v = np.random.sample(self.q)
        K = self._K()
        i = 0
        
        while  (np.linalg.norm(u, ord=1) > self.c1) and \
               (np.linalg.norm(v, ord=1) > self.c2):
            u = np.dot(K, v)
            u = self._norm(u)
            u = self._soft_thresh(u, 'u')
            u = self._norm(u)
            v = np.dot(K.T, u)
            v = self._norm(v)
            v = self._soft_thresh(v, 'v')
            v = self._norm(v)
            i += 1
            if i > 100000:
                break
        
        self.K = K
        self.u = u
        self.v = v
        self.i = i

        return self
    
    def predict(self, X_new):
        pass

In [14]:
X = np.random.random((100,10))
Y = np.random.random((100, 200))

In [15]:
test = SCCA(X,Y)

In [16]:
res=test.fit()


In [19]:
res.u

array([-0.24036635,  0.49011772,  0.38233955, -0.29974106, -0.24393519,
       -0.25918908, -0.20342491, -0.23244437, -0.20583645,  0.4489057 ])

In [20]:
res.v

array([ 0.07405644, -0.06848973, -0.06489097,  0.07457401, -0.0676253 ,
        0.07045079, -0.0680534 ,  0.07254993,  0.07203938, -0.06920278,
        0.07280125, -0.06260649, -0.06416174,  0.07261806, -0.06890614,
        0.07331063, -0.06679016, -0.06903331,  0.07252417,  0.07368516,
        0.07422394, -0.06890984,  0.07021322,  0.07793662, -0.06622216,
       -0.06972754,  0.07118383, -0.06714335,  0.07379219,  0.07810022,
       -0.06671387,  0.07535091,  0.07273917, -0.06948989,  0.0740075 ,
        0.0743911 , -0.06701335,  0.07847339,  0.07371109, -0.06560368,
        0.07889406,  0.07125198,  0.0730361 , -0.06580661,  0.07172044,
        0.07334191, -0.06244295, -0.06880922, -0.06569065,  0.07248205,
        0.07005496,  0.07338073,  0.07232826,  0.07757322,  0.07118603,
       -0.0616832 ,  0.07149578, -0.06825757,  0.07023528,  0.07011214,
        0.07549713, -0.06958255,  0.07734182, -0.06804941, -0.06423161,
        0.07399478,  0.07523319,  0.08171757, -0.05996833,  0.07